In [1]:
## model
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow.keras import layers, Model, Input
from xgboost import XGBRegressor
import joblib

# ============================
# 1. Load dataset
# ============================
df = pd.read_csv(r"C:\Users\User\Desktop\solar_code\Task\plant1\Plant1_Merged_Final.csv")

# Ensure datetime format
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])

# Time features
df['hour'] = df['DATE_TIME'].dt.hour
df['minute'] = df['DATE_TIME'].dt.minute
df['time_fraction'] = df['hour'] + df['minute']/60

# Cyclical encoding
df['sin_time'] = np.sin(2 * np.pi * df['time_fraction'] / 24)
df['cos_time'] = np.cos(2 * np.pi * df['time_fraction'] / 24)

# Drop unused columns
df = df.drop(columns=['hour','minute','time_fraction','DATE_TIME'])

# Features and target
X = df.drop(columns=['AC_POWER'])
y = df['AC_POWER'].values.reshape(-1,1)   # raw watts

# ============================
# 2. Create sliding windows
# ============================
sequence_length = 6
X_seq, y_seq = [], []

for i in range(len(X) - sequence_length):
    X_seq.append(X.iloc[i:i+sequence_length].values)
    y_seq.append(y[i+sequence_length])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq).reshape(-1,1)

# ============================
# 3. Scale X and y
# ============================
scaler_X = MinMaxScaler()
X_flat = X_seq.reshape(-1, X_seq.shape[-1])
X_scaled = scaler_X.fit_transform(X_flat).reshape(X_seq.shape)

scaler_y = MinMaxScaler()
y_scaled = scaler_y.fit_transform(y_seq)   # scaled target


In [2]:

# ============================
# 4. CNN Model
# ============================
timesteps = X_scaled.shape[1]
n_features = X_scaled.shape[2]

inputs = Input(shape=(timesteps, n_features))
x = layers.Conv1D(64, 3, activation='relu', padding="causal")(inputs)
x = layers.Conv1D(32, 3, activation='relu', padding="causal")(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(1)(x)

cnn_model = Model(inputs, outputs)
cnn_model.compile(optimizer='adam', loss='mse')

print("\nTraining CNN...")
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(patience=5, restore_best_weights=True)
history = cnn_model.fit(
    X_scaled, y_scaled,                # 🔹 target is scaled
    epochs=100,
    batch_size=16,
    validation_split=0.2,
    callbacks=[es],
    verbose=1
)

# ============================
# 5. Extract CNN features
# ============================
cnn_feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)
cnn_features = cnn_feature_extractor.predict(X_scaled)
print("Extracted CNN feature shape:", cnn_features.shape)


Training CNN...
Epoch 1/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.0121 - val_loss: 0.0069
Epoch 2/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0081 - val_loss: 0.0063
Epoch 3/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0080 - val_loss: 0.0063
Epoch 4/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0077 - val_loss: 0.0063
Epoch 5/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0076 - val_loss: 0.0060
Epoch 6/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0075 - val_loss: 0.0064
Epoch 7/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0071 - val_loss: 0.0063
Epoch 8/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0073 - val_loss: 0.0057
Epoch 9/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0073 - val_loss: 0.0060
Epoch 10/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0070 - val_loss: 0.0056
Epoch 11/100
163/163 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0066 - val_loss: 0.0067
Epoch 12/100
163/163 ━━━

In [3]:


# ============================
# 6. Train/Test split on Plant1
# ============================
X_train, X_test, y_train, y_test = train_test_split(
    cnn_features, y_scaled, test_size=0.2, random_state=42, shuffle=True
)

xgb_model = XGBRegressor(
    n_estimators=1000,
    learning_rate=0.01,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    random_state=42
)

print("\nTraining XGBoost on Plant1 (80% train, 20% test)...")
xgb_model.fit(X_train, y_train)

# ============================
# 7. Evaluate (inverse-transform back to raw y)
# ============================
y_pred_scaled = xgb_model.predict(X_test).reshape(-1,1)
y_pred = scaler_y.inverse_transform(y_pred_scaled)
y_test_raw = scaler_y.inverse_transform(y_test)

mse = mean_squared_error(y_test_raw, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_raw, y_pred)
mae = mean_absolute_error(y_test_raw, y_pred)

print(f"📊 MSE: {mse:.2f}")
print(f"📊 RMSE: {rmse:.2f}")
print(f"📊 R²: {r2:.4f}")
print(f"📊 MAE: {mae:.2f}")



Training XGBoost on Plant1 (80% train, 20% test)...
📊 MSE: 5473416.24
📊 RMSE: 2339.53
📊 R²: 0.9233
📊 MAE: 1154.23


In [4]:

# ============================
# 8. Save models & scalers
# ============================
joblib.dump(scaler_X, r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\scaler_X.pkl")
joblib.dump(scaler_y, r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\scaler_y.pkl")
cnn_model.save(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\cnn_model.h5", include_optimizer=False)
xgb_model.save_model(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\xgb_model.json")
print("✅ Models and scalers saved successfully!")


✅ Models and scalers saved successfully!


In [1]:
## inference
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import load_model, Model
from xgboost import XGBRegressor

# ============================
# Load scalers and models
# ============================
scaler_X = joblib.load(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\scaler_X.pkl")
scaler_y = joblib.load(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\scaler_y.pkl")

cnn_model = load_model(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\cnn_model.h5", compile=False)

xgb_model = XGBRegressor()
xgb_model.load_model(r"C:\Users\User\Desktop\solar_code\Task\plant1new\modely\xgb_model.json")

# Feature extractor from CNN
cnn_feature_extractor = Model(inputs=cnn_model.input, outputs=cnn_model.layers[-2].output)

# ============================
# Preprocessing (CSV → windows)
# ============================
def preprocess_and_window(df, sequence_length=6):
    df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'])
    df['hour'] = df['DATE_TIME'].dt.hour
    df['minute'] = df['DATE_TIME'].dt.minute
    df['time_fraction'] = df['hour'] + df['minute']/60

    # cyclical encoding
    df['sin_time'] = np.sin(2*np.pi*df['time_fraction']/24)
    df['cos_time'] = np.cos(2*np.pi*df['time_fraction']/24)

    # drop unused
    df = df.drop(columns=['hour','minute','time_fraction','DATE_TIME'])

    X = df.drop(columns=['AC_POWER'])
    y = df['AC_POWER'].values

    # sliding windows
    X_seq, y_seq = [], []
    for i in range(len(X)-sequence_length):
        X_seq.append(X.iloc[i:i+sequence_length].values)
        y_seq.append(y[i+sequence_length])

    X_seq = np.array(X_seq)
    y_seq = np.array(y_seq)

    # scale features
    X_flat = X_seq.reshape(-1, X_seq.shape[-1])
    X_scaled = scaler_X.transform(X_flat).reshape(X_seq.shape)

    return X_scaled, y_seq


# ============================
# General Prediction Function
# ============================
def predict_ac_power(test_csv, sequence_length=6, num_windows=None):
    """
    test_csv: path to CSV
    sequence_length: sliding window length
    num_windows: None → use full CSV, int → use first N windows
    """
    df_test = pd.read_csv(test_csv)
    X_scaled, y_true = preprocess_and_window(df_test, sequence_length)

    if num_windows is not None:
        X_scaled = X_scaled[:num_windows]
        y_true = y_true[:num_windows]

    # CNN → XGB
    cnn_features = cnn_feature_extractor.predict(X_scaled)
    y_pred_scaled = xgb_model.predict(cnn_features).reshape(-1,1)

    # inverse-transform predictions to raw watts
    y_pred = scaler_y.inverse_transform(y_pred_scaled)

    return y_true, y_pred.flatten()


# ============================
if __name__ == "__main__":
    test_csv = r"C:\Users\User\Desktop\solar_code\Task\plant1\Plant1_Merged_Final.csv"

    # 1. Full CSV Prediction
    y_true, y_pred = predict_ac_power(test_csv)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print("✅ Full Plant1 Inference Complete")
    print(f"MSE : {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAE : {mae:.2f}")
    print(f"R²  : {r2:.4f}")

    # Show first 10 predictions
    print("\n🔹 First 10 Predictions:")
    for i in range(10):
        print(f"True: {y_true[i]:.2f} W | Predicted: {y_pred[i]:.2f} W")

    # 2. Predict only first 1 window
    y_true_1, y_pred_1 = predict_ac_power(test_csv, num_windows=1)
    print("\n🔹 One Window Example")
    print(f"True Next AC Power: {y_true_1[0]:.2f} W | Predicted: {y_pred_1[0]:.2f} W")

    # 3. Predict first 60 windows
    y_true_60, y_pred_60 = predict_ac_power(test_csv, num_windows=60)
    print(f"\n🔹 First 60 windows → got {len(y_true_60)} predictions")


102/102 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
✅ Full Plant1 Inference Complete
MSE : 1445840.83
RMSE: 1202.43
MAE : 527.49
R²  : 0.9802

🔹 First 10 Predictions:
True: 0.00 W | Predicted: 70.08 W
True: 0.00 W | Predicted: 59.78 W
True: 0.00 W | Predicted: 57.31 W
True: 0.00 W | Predicted: 57.31 W
True: 0.00 W | Predicted: 26.86 W
True: 0.00 W | Predicted: 26.86 W
True: 0.00 W | Predicted: 26.49 W
True: 0.00 W | Predicted: 26.49 W
True: 0.00 W | Predicted: 26.49 W
True: 0.00 W | Predicted: 30.80 W
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step

🔹 One Window Example
True Next AC Power: 0.00 W | Predicted: 70.08 W
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step

🔹 First 60 windows → got 60 predictions
